# Incremento de frecuencia sin datos de pozos 3D

* [Douglas Alberto Gómez Reyes](agomezr@imp.mx)- Dirección de Servicios en Exploración y Producción
* [Juan Delfino Lara Corona](jdlara@imp.mx)- Dirección de Servicios en Exploración y Producción
* [Liliana Vargas Meleza](lvargasm@imp.mx)- Dirección de Servicios en Exploración y Producción
* [Sergio Chávez Pérez](sergio.chavezp@gmail.com)- Dirección de Tecnología de Producto

    **Instituto Mexicano del Petróleo**

### Introducción

La resolución sísmica vertical es función del contenido de frecuencia, por lo que frecuentemente
surgen nuevas técnicas para incrementar el ancho de banda de datos sísmicos. Esto es factible
mediante la manipulación espectral de los datos, que involucra conceptos fundamentales de
procesamiento de señales, indispensables para realizar actividades cotidianas en las áreas de
exploración y producción petroleras. El contenido de energía de los datos sísmicos está representado
en el espectro de amplitud y es de banda limitada en frecuencia, por lo que es deseable incrementar
el ancho de banda para fines de incremento de resolución sísmica y de estabilización de operaciones
de filtrado y de procesamiento de datos. En este trabajo ilustramos la relevancia y sencillez de la
manipulación espectral, usando herramientas de código abierto, revisitando técnicas de incremento
de frecuencia en datos sísmicos mediante cuadernos de trabajo y reproducibilidad computacional,
para el incremento de frecuencia.

Cotidianamente, en el ambiente académico e industrial, los procesadores e intérpretes utilizan
diferentes métodos de manipulación espectral de datos sísmicos (e.g., [Silvia, 1987](https://www.qsl.net/k/kb1uif//downloads/Docs/ELLIOTT,%20D.%20F.%20(1987)%20Handbook%20of%20Digital%20Signal%20Processing%20-%20Engineering%20Applications.pdf); [Claerbout, 1992](http://sep.stanford.edu/sep/prof/pvi.pdf))
entre los que destacan el blanqueamiento, azulamiento (incremento de frecuencias altas), balanceo,
factor de amortiguamiento en inversión lineal y deconvolución (o remoción de efectos de filtrado). En
el balanceo espectral de datos sísmicos (e.g., [Chopra y Marfurt, 2016](http://www.chopraseismic.com/wp-content/uploads/2016/02/3502_TSA_IC_Chopra-resized.pdf)) interviene un factor de
amortiguamiento que equivale a reforzar la diagonal principal de la matriz utilizada en el proceso de
inversión lineal y en el blanqueamiento del espectro. Al encontrar dicho factor (mediante las
ecuaciones normales) podemos establecer su equivalencia con el balanceo espectral.

Nuestro interés actual yace en la mejoría de flujos de trabajo y prácticas de interpretación
sísmica, con herramientas computacionales reproducibles y no comerciales, y hemos enfocado
nuestra atención en el surgimiento y uso de cuadernos de trabajo (e.g., [Osipov, 2016](https://www.datanami.com/2016/05/04/rise-data-science-notebooks/?eid=328382723&bid=1398456); [Kluyver et al.,
2016](http://microblogging.infodocs.eu/wp-content/uploads/2016/06/9781614996491.pdf)), que representan un formato de publicación para ilustrar flujos de trabajo reproducibles, realizados con paquetería gratuita y de código abierto, y con recursos computacionales que están al alcance de cualquier estudiante, académico o profesional.

[Kluyver et al. (2016)](http://microblogging.infodocs.eu/wp-content/uploads/2016/06/9781614996491.pdf) destacan que los cuadernos de trabajo (nuestra traducción de notebooks),
documentos que integran prosa, código y resultados, ofrecen una manera de publicar un método
computacional que puede ser fácilmente leído y replicado. Utilizamos esa forma de trabajo para
presentar y replicar nuestros resultados

### Instrucciones

Las técnicas descritas como I y II, las ejecutamos con la paquetería libre de código abierto [Madagascar](http://www.ahay.org/wiki/Main_Page). La técnica III, la realizamos con [AASPI](http://mcee.ou.edu/aaspi/). 

Madagascar es una paquetería que integra elementos de sus precursores [SEPLib](http://sepwww.stanford.edu/) y [Seismic Unix](http://www.cwp.mines.edu/cwpcodes/) empleando esquemas de cómputo moderno. La paquetería consta de tres niveles: 
> 1) Programas de procesamiento: Más de 750 programas desarrollados principalmente en lenguaje C (también en C++, Fortran, Python, etc.) que actúan como filtros sobre los datos y pueden encadenarse mediante “pipes” de Unix o bien cuadernos de trabajo de Jupyter (http://jupyter.org/); 

> 2) Flujos de procesamiento SConstruct: Escritos en lenguaje de programación Python y ejecutados mediante el programa SCons; 

>3) Documentos reproducibles: conjunto de libros y artículos técnicos. 

[AASPI](http://mcee.ou.edu/aaspi/) es la paquetería de código abierto desarrollada por el consorcio académico de investigación Attribute-Assisted Seismic Processing and Interpretacion de la Universidad de Oklahoma, EEUU.


In [ ]:
from m8r import view 
import math

### Lectura de datos sísmicos

Probamos los flujos de trabajo de las técnicas de incremento de frecuencia sin datos de pozos, en tres dimensiones, con datos sísmicos marinos, libres, propiedad del Departamento de Energía de Nueva Escocia y distribuido por la compañía [`dGB Earth Sciences`](https://dgbes.com/) bajo una licencia [CC BY-SA]() (Fig.1), y disponibles [aquí](https://opendtect.org/osr/pmwiki.php/Main/PENOBSCOT3DSABLEISLAND).

In [ ]:
%%file carga.scons 

sis_ori = 'penobscot_ori500_3000.sgy'

Flow('seudo2d', sis_ori, 'segyread tfile=tfile.rsf bfile=bfile hfile=hfile verb=y endian=y')
Flow('cubo', 'seudo2d', '''intbin inv=n head=tfile.rsf xk=xline yk=iline | put label1=Tiempo 
                                    label3=inline label2=xline | window  min1=0.5 max1=3''')

In [ ]:
%%file display_input.scons

Plot('cubo-ori', 'cubo', '''byte gainpanel=all | grey3 frame1=550 frame2=100 frame3=100 
       title="Datos originales" flat=n point1=0.8 point2=0.8 point3=0 color=i''' )
Result('original', 'cubo-ori', 'SideBySideAniso' )

In [ ]:
view('original')

<font size=2>**Figura 1.** Datos sísmicos originales, recortados en tiempo de 0.5 a 3.0 segundos.

## Derivadas orden superior

### Técnica I: Negativo de la segunda derivada y cuarta derivada

El negativo de la segunda derivada permite mejorar la definición de los datos sísmicos recorriendo el contenido de frecuencia hacia las frecuencias altas. En el dominio de las frecuencias, la derivada implica un cambio en la amplitud y frecuencia, proporcional a la amplitud y frecuencia originales, lo que produce un efecto de filtro pasa altas, recorriendo la frecuencia predominante hacia las frecuencias altas. La cuarta derivada recorre aún más las frecuencias hacia las altas frecuencias.

Derivar una señal $f(t)$ equivale, en el dominio de la frecuencia, a multiplicar la transformada de Fourier de la señal $F(ω)$ por el producto $iω$, donde $i$ es la unidad imaginaria y $ω$ es la frecuencia angular. Derivar dos veces equivale a multiplicar $F(ω)$ por $(iω)²$ , es decir, por $−ω ²$ . Este cambio de signo implica un cambio en la polaridad de la traza, por lo que es necesario multiplicar por −1 para recuperar la polaridad original.

La Fig. 2 muestra el flujo de trabajo que implementamos en Madgascar.

<img src="derv1_wf.png" alt="drawing" style="float:left;width:480px;height:240px;border:left;"/>

<font size=2>**Figura 2.** Flujo de trabajo de derivadas de orden superior.

In [ ]:
%%file der.scons 

Flow('der2', 'cubo', 'deriv | deriv | scale dscale=-1 ')
Flow('der4', 'cubo', 'deriv | deriv | deriv | deriv ')

In [ ]:
%%file display_a.scons 

Plot('cubo-deriv2', 'der2', '''byte gainpanel=all | grey3 frame1=550 frame2=100 frame3=100 
           title="2da derivada" flat=n point1=0.8 point2=0.8 point3=0 color=i ''' )
Plot('cubo-deriv4', 'der4', ''' byte gainpanel=all | grey3 frame1=550 frame2=100 frame3=100 
           title="4ta derivada" flat=n point1=0.8 point2=0.8 point3=0 color=i ''' )
           
Result('derv', 'cubo-ori cubo-deriv2 cubo-deriv4', 'SideBySideAniso')

### Resultados 3D

La Fig. 3 muestra resultados de las pruebas en los datos sísmicos 3D.

In [ ]:
view('derv')

<font size=2>**Figura 3.** **Izquierda**. Datos sísmicos originales. **Centro** Negativo de la segunda derivada. **Derecha**. Cuarta derivada.

## Mutiplicador de fase

### Técnica II: Multiplicador de fase

Stark (2009) sugirió una estrategia para mejorar resolución vertical al incrementar el número de valles y cimas de trazas sísmicas incorporando un multiplicador N que acelere la fase instantánea. 

La técnica del multiplicador de fase está basada en la separación de la información de amplitud y de fase en una traza sísmica, dada por la ecuación,

> $f(t)=A(t)cos(\theta(t))$ 

donde $x(t)$ es una traza sísmica, $A(t)$ es la amplitud instantánea y $\theta(t)$ es la fase instantánea. 

La técnica consiste únicamente en agregar un multiplicador $N$ en la ecuación anterior

> $f(t)=A(t)cos(N\theta(t))$ 

El valor del multiplicador toma valores enteros (1 a 5, usualmente) y su efecto consiste en acelerar la fase, incrementando así la frecuencia. Otros resultados útiles se obtienen al sumar los multiplicadores de fase nones (N=1+N=3 y N=1+N=3+N=5).

La Fig. 4 muestra el flujo que implementamos.

<img src="mult1_wf.png" alt="drawing" style="float:left;width:360px;height:250px;border:left;"/>

<font size=2>**Figura 4.** Flujo de trabajo del multiplicador de fase.

In [ ]:
%%file mult.scons 

Flow('interp', 'cubo', 'sinc n1=1252 d1=0.002 o1=0.5')
Flow('cuad', 'interp', 'envelope hilb=y')
Flow('amp', None, 'math a=interp.rsf b=cuad.rsf output="sqrt(a^2 + b^2)"')
Flow('phase', None, 'math a=interp.rsf b=cuad.rsf output="atan(b/a)"')
Flow('mult2-inter', None, 'math a=amp.rsf b=phase.rsf output="a*cos(2*b)"')
Flow('mult3-inter', None, 'math a=amp.rsf b=phase.rsf output="a*cos(3*b)"')
Flow('sumn1n3-inter', None, 'math a=interp.rsf b=mult3-inter.rsf output="a+b"')
Flow('mult2', 'mult2-inter', 'sinc n1=626 d1=0.004 o1=0.5')
Flow('mult3', 'mult3-inter', 'sinc n1=626 d1=0.004 o1=0.5')
Flow('sumn1n3', 'sumn1n3-inter', 'sinc n1=626 d1=0.004 o1=0.5')

In [ ]:
%%file display_b.scons 

Plot('cubo-mult2', 'mult2', '''byte gainpanel=all | grey3 frame1=550 frame2=100 frame3=100 
               title="Multiplicador de fase N2" flat=n point1=0.8 point2=0.8 point3=0 color=i ''' )

Plot('cubo-mult3', 'mult3', '''byte gainpanel=all | grey3 frame1=550 frame2=100 frame3=100 
               title="e) Multiplicador de fase N3" flat=n point1=0.8 point2=0.8 point3=0 color=i ''' )

Plot('cubo-sumn1n3', 'sumn1n3', '''byte gainpanel=all | grey3 frame1=550 frame2=100 frame3=100 
               title="Multiplicador de fase N1+N3" flat=n point1=0.8 point2=0.8 point3=0 color=i ''' )

Result('mult', 'cubo-mult2 cubo-mult3 cubo-sumn1n3', 'SideBySideAniso' )

Result('derv_mult', 'cubo-ori cubo-deriv2 cubo-deriv4 cubo-mult2 cubo-mult3 cubo-sumn1n3', 'TwoRows' )

### Resultados 3D

La Fig. 5 muestra resultados de las pruebas en los datos sísmicos 3D.

In [ ]:
view('mult')

<font size=2>**Figura 5.**  **Izquierda**. Datos sísmicos originales. **Centro**. Multiplicador N3. **Derecha**. Multiplicador N1 + N3.

### Resultados en sección 

Las Fig. 6 y Fig. 7 muestran resultados de las pruebas de las derivadas de orden superior y multiplicador de fase en la línea longitudinal 1177.

In [ ]:
%%file display_c.scons 

Flow('il1210-ori', 'cubo', 
                '''window min1=1.6 max1=2.6 max2=1300 min3=1177 n3=1''')
Flow('il1210-der2', 'der2', 
                '''window min1=1.6 max1=2.6 max2=1300 min3=1177 n3=1''')
Flow('il1210-der4', 'der4', 
                '''window min1=1.6 max1=2.6 max2=1300 min3=1177 n3=1''')
Flow('il1210_mult2', 'mult2', 
                '''window min1=1.6 max1=2.6 max2=1300 min3=1177 n3=1''')
Flow('il1210_mult3', 'mult3', 
                '''window min1=1.6 max1=2.6 max2=1300 min3=1177 n3=1''')
Flow('il1210_sumn1n3', 'sumn1n3', 
                '''window min1=1.6 max1=2.6 max2=1300 min3=1177 n3=1''')
##########
Plot('seis', 'il1210-ori',
            'gaussmooth rect=3 | grey gainpanel=a  title="Datos Originales" color=g ')
Plot('der2', 'il1210-der2',
            'gaussmooth rect=3 | grey gainpanel=a  title="2da derivada" color=g ')
Plot('der4', 'il1210-der4',
            'gaussmooth rect=3 | grey gainpanel=a  title="4da derivada" color=g ')
##########                            
Plot('mult2', 'il1210_mult2',
            'gaussmooth rect=3 | grey gainpanel=a  title="Multiplicador N2"  color=g')
Plot('mult3', 'il1210_mult3',
            'gaussmooth rect=3 | grey gainpanel=a  title="Multiplicador N3"  color=g')
Plot('sumn1n3', 'il1210_sumn1n3',
            'gaussmooth rect=3 | grey gainpanel=a  title="Multiplicador N1+N3" color=g')
##########
Result('seis2-der', 'seis der2 der4', 'SideBySideAniso')
Result('seis2-mult', 'seis mult2 sumn1n3', 'SideBySideAniso')

In [ ]:
view('seis2-der')

<font size=2>**Figura 6.**  **Izquierda**. Datos sísmicos originales. **Centro**. Negativo de la segunda derivada. **Derecha**. Cuarta derivada.

In [ ]:
view('seis2-mult')

<font size=2>**Figura 7.**  **Izquierda**. Datos sísmicos originales. **Centro**. Multiplicador N2. **Derecha**. Multiplicador N1 + N3.

### Efecto del incremento de frecuencia

La Fig. 8 compara el efecto del incremento de frecuencia en los espectros de amplitud de cada una de las pruebas.

<img src="espec_pot_dermult1.png" alt="drawing" style="float:left;width:680px;height:250px;border:left;"/>

<font size=2>**Figura 8.**  **Izquierda**. Efecto en el espectro de amplitud al incrementar la frecuencia derivando.  **Derecha**. Efecto en el espectro de amplitud al incrementar la frecuencia multiplicando la fase.

La Fig. 9 muestra resultados de técnica I en la vecindad del pozo L-30. El recuadro de color amarillo muestra rasgos revelados por el operador derivada (2da y 4ta) y validados con el sismograma sintético.

<img src="derv2_wf.png" alt="drawing" style="float:left;width:920px;height:710px;border:left;"/>

<font size=2>**Figura 9.**  **Izquierda**. Datos sísmicos originales. **Centro**. Negativo de la segunda derivada. **Derecha**. Cuarta derivada. Los recuadros de color amarillo muestran eventos sísmicos resueltos que no eran distinguibles en los datos originales (recuadro de color blanco).

La Fig. 10 muestra resultados de la técnica II en la vecindad del pozo L-30. El recuadro de color amarillo muestra rasgos revelados por los multiplicadores de fase (N2 y N1 + N3) y validados con el sismograma sintético.

<img src="mult2_wf.png" alt="drawing" style="float:left;width:920px;height:710px;border:left;"/>

<font size=2>**Figura 10.**  **Izquierda**. Datos sísmicos originales. **Centro**. Multiplicador de fase N2. **Derecha**. Suma de multiplicadores N1 + N3. Los recuadros de color amarillo muestran eventos sísmicos resueltos que no eran distinguibles en los datos originales (recuadro de color blanco). Esta técnica, con respecto a la técnica I, ofrece mejorías inferiores de resolución vertical.

### Técnica III: Filtrado orientado a estructura (FOE) + balanceo espectral

Al aplicar filtrado orientado a estructura (FOE) en distintas bandas de frecuencia, junto con balanceo espectral, es posible aumentar la relación señal a ruido fuera de la banda central original y, al mismo tiempo, mejorar el contenido de frecuencia de los datos (Helmore, 2009). La Fig. 11 muestra el flujo de trabajo implementado.

<img src="foe_bal1.png" alt="drawing" style="float:left;width:380px;height:290px;border:left;"/>

**Figura 11.** Flujo de trabajo de FOE con balanceo espectral.

La Fig. 12 muestra los resultados de la técnica III en la vecindad del pozo L-30. El recuadro de color amarillo muestra rasgos revelados por FOE + balanceo espectral y validados con sismogramas sintéticos de los pozos en cuestión.

<img src="foe_bal2.png" alt="drawing" style="float:left;width:640px;height:640px;border:left;"/>

La Fig. 15 muestra espectros de frecuencia obtenidos con cada una de las técnicas.

<img src="espec_pot_dermult2.png" alt="drawing" style="float:left;width:1160px;height:240px;border:left;"/>

**Figura 15.** Espectros de amplitud de datos originales versus las técnicas de incremento de frecuencia implementadas.

In [ ]:
#!scons -cQ